## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Oct. 13th Oct. 20th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,October-15-2024,Condo/Co-op,1450 N Riverside Dr #204,Pompano Beach,FL,33062.0,170000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1450-N...,Beaches MLS,F10464994,N,Y,26.250937,-80.088453


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
39,PAST SALE,2024-10-21,Condo/Co-op,7300 NW 5th Pl #105,Margate,FL,33063.0,86000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Margate/7300-NW-5th-...,Beaches MLS,F10456983,N,Y,26.237593,-80.222934
56,PAST SALE,2024-10-18,Condo/Co-op,4100 Crystal Lake Dr #206,Deerfield Beach,FL,33064.0,120000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/4100...,MARMLS,A11620297,N,Y,26.281285,-80.143384
28,PAST SALE,2024-10-18,Condo/Co-op,3200 NE 36th St #215,Fort Lauderdale,FL,33308.0,345000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Beaches MLS,F10459536,N,Y,26.171862,-80.102789
97,PAST SALE,2024-10-18,Condo/Co-op,300 Oregon St #506,Hollywood,FL,33019.0,380000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/300-Oregon...,Beaches MLS,F10431028,N,Y,26.008556,-80.116063
33,PAST SALE,2024-10-18,Condo/Co-op,1407 NE 56th St #105,Fort Lauderdale,FL,33334.0,250000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1407...,Beaches MLS,F10457950,N,Y,26.196706,-80.130777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,PAST SALE,2024-10-15,Condo/Co-op,2907 NE 8th Ter #105,Oakland Park,FL,33334.0,283500.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Wilton-Manors/2907-N...,Beaches MLS,F10436137,N,Y,26.164554,-80.135771
89,PAST SALE,2024-10-15,Condo/Co-op,800 Parkview Dr #126,Hallandale Beach,FL,33009.0,180000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/800...,MARMLS,A11556620,N,Y,25.995195,-80.121005
38,PAST SALE,2024-10-15,Condo/Co-op,9575 Weldon Cir #305,Tamarac,FL,33321.0,200000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Tamarac/9575-Weldon-...,Beaches MLS,F10453206,N,Y,26.221008,-80.277963
1,PAST SALE,2024-10-15,Condo/Co-op,1450 N Riverside Dr #204,Pompano Beach,FL,33062.0,170000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1450-N...,Beaches MLS,F10464994,N,Y,26.250937,-80.088453


In [10]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [11]:
df_top_ten = df.head(10)

## Collect Agent Information

In [12]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [13]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [14]:
response_df = pd.DataFrame(response_list)

In [15]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [16]:
df_top_ten = merged_df

## Current Week's Values

In [17]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

111


In [18]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$302,175


In [19]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$276


In [20]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$33,541,399


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [21]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Broward
Input Previous Week Condo Sales Total (number units sold):
70
Input Previous Week Condo Average Sales Price:
294649
Input Previous Week Condo Average PSF:
253
Input Previous Week Condo Sales total (ex: 198_000_000)
20_600_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [29]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [30]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [31]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [32]:
pd.set_option('display.max_columns',None)

In [33]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-18,Condo/Co-op,730 N Ocean Blvd #505,Pompano Beach,FL,33062.0,1720000.0,3.0,3.0,SABBIA BEACH CONDO,1830.0,NaN,2019.0,NaN,940.0,2997.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/730-N-...,MARMLS,A11619734,N,Y,26.243932,-80.087321,https://www.redfin.com/FL/Pompano-Beach/730-N-...,Daniel Mahaney,RealLook.com,None,None,None,None,None,None,1,orange


In [34]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [35]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-18,Condo/Co-op,730 N Ocean Blvd #505,Pompano Beach,FL,33062.0,1720000.0,3.0,3.0,SABBIA BEACH CONDO,1830.0,NaN,2019.0,NaN,940.0,2997.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/730-N-...,MARMLS,A11619734,N,Y,26.243932,-80.087321,https://www.redfin.com/FL/Pompano-Beach/730-N-...,Daniel Mahaney,RealLook.com,None,None,None,None,None,None,1,orange
1,PAST SALE,2024-10-15,Condo/Co-op,100 E Las Olas Blvd #1701,Fort Lauderdale,FL,33301.0,1625000.0,3.0,2.5,100 E Las Olas,2395.0,NaN,2020.0,NaN,678.0,2792.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Beaches MLS,F10455449,N,Y,26.118835,-80.141476,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Peter Barkin,"Compass Florida, LLC",Johnny Ramirez,"Compass Florida, LLC",Adriana Resendez,Coldwell Banker Realty,None,None,2,blue
2,PAST SALE,2024-10-18,Condo/Co-op,3400 Galt Ocean Dr Unit 501S,Fort Lauderdale,FL,33308.0,1212500.0,2.0,2.5,SOUTHPOINT CONDO,1891.0,NaN,1974.0,NaN,641.0,1468.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3400...,MARMLS,A11657484,N,Y,26.169887,-80.098341,https://www.redfin.com/FL/Fort-Lauderdale/3400...,Sara Arroyo,"Compass Florida, LLC",Megan Probst,"Compass Florida, LLC",None,None,None,None,3,blue
3,PAST SALE,2024-10-17,Condo/Co-op,6001 N Ocean Dr #702,Hollywood,FL,33019.0,940000.0,2.0,2.5,RENAISSANCE ON THE OCEAN,2025.0,NaN,1999.0,NaN,464.0,2316.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/6001-N-Oce...,MARMLS,A11584064,N,Y,26.051895,-80.112743,https://www.redfin.com/FL/Hollywood/6001-N-Oce...,Nina Spinner-Sands,The Keyes Company,Rita Morse,The Keyes Company,Florencia Baques Landry,"BB Realty Group, LLC.",None,None,4,blue
4,PAST SALE,2024-10-18,Condo/Co-op,3111 N Ocean Dr #1107,Hollywood,FL,33019.0,925000.0,3.0,2.0,Hollywood Towers,1650.0,NaN,1967.0,NaN,561.0,1214.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3111-N-Oce...,Beaches MLS,F10455532,N,Y,26.030688,-80.115514,https://www.redfin.com/FL/Hollywood/3111-N-Oce...,Lilli Schipper,LoKation,None,None,Rachael Osburn,Sunshine Realty,None,None,5,blue
5,PAST SALE,2024-10-16,Condo/Co-op,3100 NE 48th St Ph 13,Fort Lauderdale,FL,33308.0,700000.0,2.0,2.0,Pilot House,1544.0,NaN,1977.0,NaN,453.0,1138.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10413857,N,Y,26.188219,-80.103988,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Timothy Ryan,Galt Ocean Realty Inc,None,None,Timothy Ryan,Galt Ocean Realty Inc,None,None,6,blue
6,PAST SALE,2024-10-18,Condo/Co-op,4111 S Ocean Dr #705,Hollywood,FL,33019.0,635000.0,1.0,1.0,4111 SOUTH OCEAN DRIVE CO,701.0,NaN,2017.0,NaN,906.0,955.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/4111-S-Oce...,MARMLS,A11573649,N,Y,25.986806,-80.118650,https://www.redfin.com/FL/Hollywood/4111-S-Oce...,Jennifer Leong,One Sotheby's Int'l Realty,Mareste Contreras,One Sotheby's Int'l Realty,Mareste Contreras,One Sotheby's Int'l Realty,None,None,7,blue
7,PAST SALE,2024-10-16,Condo/Co-op,4111 S Ocean Dr #1412,Hollywood,FL,33019.0,619000.0,1.0,1.0,4111 SOUTH OCEAN DRIVE CO,869.0,NaN,2017.0,NaN,712.0,1191.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/4111-S-Oce...,MARMLS,A11530514,N,Y,25.986806,-80.118650,https://www.redfin.com/FL/Hollywood/4111-S-Oce...,Ricardo Schilkrut Cogan,One Sotheby's International Realty,None,None,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,8,blue
8,PAST SALE,2024-10-15,Condo/Co-op,16100 Emerald Estates Dr #483,Weston,FL,33331.0,590000.0,2.0,2.5,WESTON 55 PLUS CONDO,1594.0,NaN,2006.0,NaN,370.0,1332.0,Sold,NaN,NaN,https://www.redfin

In [36]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [37]:
m.save('index.html')

## Data snagger

In [38]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [39]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-18,Condo/Co-op,730 N Ocean Blvd #505,Pompano Beach,FL,33062.0,1720000.0,3.0,3.0,SABBIA BEACH CONDO,1830.0,NaN,2019.0,NaN,940.0,2997.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/730-N-...,MARMLS,A11619734,N,Y,26.243932,-80.087321,https://www.redfin.com/FL/Pompano-Beach/730-N-...,Daniel Mahaney,RealLook.com,None,None,None,None,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [40]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Broward_County_condo_sales_week_ending_10212024


# CREATE TEMPLATE 

In [41]:
muni_set = set(df_top_ten['CITY'])

In [42]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [43]:
df_top_ten.reset_index(inplace=True,drop=True)

In [44]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [45]:
top_sale

'SABBIA BEACH CONDO at 730 N Ocean Blvd #505 in Pompano Beach'

In [46]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [47]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [48]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-18,Condo/Co-op,730 N Ocean Blvd #505,Pompano Beach,FL,33062.0,1720000.0,3.0,3.0,SABBIA BEACH CONDO,1830.0,NaN,2019.0,NaN,940.0,2997.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/730-N-...,MARMLS,A11619734,N,Y,26.243932,-80.087321,https://www.redfin.com/FL/Pompano-Beach/730-N-...,Daniel Mahaney,RealLook.com,None,None,None,None,None,None,1,orange,SABBIA BEACH CONDO at 730 N Ocean Blvd #505 in...
1,PAST SALE,2024-10-15,Condo/Co-op,100 E Las Olas Blvd #1701,Fort Lauderdale,FL,33301.0,1625000.0,3.0,2.5,100 E Las Olas,2395.0,NaN,2020.0,NaN,678.0,2792.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Beaches MLS,F10455449,N,Y,26.118835,-80.141476,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Peter Barkin,"Compass Florida, LLC",Johnny Ramirez,"Compass Florida, LLC",Adriana Resendez,Coldwell Banker Realty,None,None,2,blue,100 E Las Olas at 100 E Las Olas Blvd #1701 in...
2,PAST SALE,2024-10-18,Condo/Co-op,3400 Galt Ocean Dr Unit 501S,Fort Lauderdale,FL,33308.0,1212500.0,2.0,2.5,SOUTHPOINT CONDO,1891.0,NaN,1974.0,NaN,641.0,1468.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3400...,MARMLS,A11657484,N,Y,26.169887,-80.098341,https://www.redfin.com/FL/Fort-Lauderdale/3400...,Sara Arroyo,"Compass Florida, LLC",Megan Probst,"Compass Florida, LLC",None,None,None,None,3,blue,SOUTHPOINT CONDO at 3400 Galt Ocean Dr Unit 50...
3,PAST SALE,2024-10-17,Condo/Co-op,6001 N Ocean Dr #702,Hollywood,FL,33019.0,940000.0,2.0,2.5,RENAISSANCE ON THE OCEAN,2025.0,NaN,1999.0,NaN,464.0,2316.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/6001-N-Oce...,MARMLS,A11584064,N,Y,26.051895,-80.112743,https://www.redfin.com/FL/Hollywood/6001-N-Oce...,Nina Spinner-Sands,The Keyes Company,Rita Morse,The Keyes Company,Florencia Baques Landry,"BB Realty Group, LLC.",None,None,4,blue,RENAISSANCE ON THE OCEAN at 6001 N Ocean Dr #7...
4,PAST SALE,2024-10-18,Condo/Co-op,3111 N Ocean Dr #1107,Hollywood,FL,33019.0,925000.0,3.0,2.0,Hollywood Towers,1650.0,NaN,1967.0,NaN,561.0,1214.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3111-N-Oce...,Beaches MLS,F10455532,N,Y,26.030688,-80.115514,https://www.redfin.com/FL/Hollywood/3111-N-Oce...,Lilli Schipper,LoKation,None,None,Rachael Osburn,Sunshine Realty,None,None,5,blue,Hollywood Towers at 3111 N Ocean Dr #1107 in H...
5,PAST SALE,2024-10-16,Condo/Co-op,3100 NE 48th St Ph 13,Fort Lauderdale,FL,33308.0,700000.0,2.0,2.0,Pilot House,1544.0,NaN,1977.0,NaN,453.0,1138.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10413857,N,Y,26.188219,-80.103988,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Timothy Ryan,Galt Ocean Realty Inc,None,None,Timothy Ryan,Galt Ocean Realty Inc,None,None,6,blue,Pilot House at 3100 NE 48th St Ph 13 in Fort L...
6,PAST SALE,2024-10-18,Condo/Co-op,4111 S Ocean Dr #705,Hollywood,FL,33019.0,635000.0,1.0,1.0,4111 SOUTH OCEAN DRIVE CO,701.0,NaN,2017.0,NaN,906.0,955.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/4111-S-Oce...,MARMLS,A11573649,N,Y,25.986806,-80.118650,https://www.redfin.com/FL/Hollywood/4111-S-Oce...,Jennifer Leong,One Sotheby's Int'l Realty,Mareste Contreras,One Sotheby's Int'l Realty,Mareste Contreras,One Sotheby's Int'l Realty,None,None,7,blue,4111 SOUTH OCEAN DRIVE CO at 4111 S Ocean Dr #...
7,PAST SALE,2024-10-16,Condo/Co-op,4111 S Ocean Dr #1412,Hollywood,FL,33019.0,619000.0,1.0,1.0,4111 SOUTH OCEAN DRIVE CO,869.0,NaN,2017.0,NaN,712.0,1191.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/4111-S-Oce...,MARMLS,A11530514,N,Y,25.986806,-80.1186

In [49]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: SABBIA BEACH CONDO closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $575,000M to $1,720,000M
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Broward condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Fort Lauderdale, Pompano Beach, Hollywood, Weston

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 111 condo sales totaling $33,541,399 million from Oct. 13th to Oct. 20th. The previous week, brokers closed 70 condo sales totaling $20,600,000.

Last week’s units sold for an average of $302,175, higher than the $294,649 average price from the previous week. The average price per square foot rose to $276

In [50]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [51]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [52]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [53]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-18,Condo/Co-op,730 N Ocean Blvd #505,Pompano Beach,FL,33062.0,1720000.0,3.0,3.0,SABBIA BEACH CONDO,1830.0,NaN,2019.0,NaN,940.0,2997.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/730-N-...,MARMLS,A11619734,N,Y,26.243932,-80.087321,https://www.redfin.com/FL/Pompano-Beach/730-N-...,Daniel Mahaney,RealLook.com,None,None,None,None,None,None,1,orange,SABBIA BEACH CONDO at 730 N Ocean Blvd #505 in...
1,PAST SALE,2024-10-15,Condo/Co-op,100 E Las Olas Blvd #1701,Fort Lauderdale,FL,33301.0,1625000.0,3.0,2.5,100 E Las Olas,2395.0,NaN,2020.0,NaN,678.0,2792.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Beaches MLS,F10455449,N,Y,26.118835,-80.141476,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Peter Barkin,"Compass Florida, LLC",Johnny Ramirez,"Compass Florida, LLC",Adriana Resendez,Coldwell Banker Realty,None,None,2,blue,100 E Las Olas at 100 E Las Olas Blvd #1701 in...
2,PAST SALE,2024-10-18,Condo/Co-op,3400 Galt Ocean Dr Unit 501S,Fort Lauderdale,FL,33308.0,1212500.0,2.0,2.5,SOUTHPOINT CONDO,1891.0,NaN,1974.0,NaN,641.0,1468.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3400...,MARMLS,A11657484,N,Y,26.169887,-80.098341,https://www.redfin.com/FL/Fort-Lauderdale/3400...,Sara Arroyo,"Compass Florida, LLC",Megan Probst,"Compass Florida, LLC",None,None,None,None,3,blue,SOUTHPOINT CONDO at 3400 Galt Ocean Dr Unit 50...
3,PAST SALE,2024-10-17,Condo/Co-op,6001 N Ocean Dr #702,Hollywood,FL,33019.0,940000.0,2.0,2.5,RENAISSANCE ON THE OCEAN,2025.0,NaN,1999.0,NaN,464.0,2316.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/6001-N-Oce...,MARMLS,A11584064,N,Y,26.051895,-80.112743,https://www.redfin.com/FL/Hollywood/6001-N-Oce...,Nina Spinner-Sands,The Keyes Company,Rita Morse,The Keyes Company,Florencia Baques Landry,"BB Realty Group, LLC.",None,None,4,blue,RENAISSANCE ON THE OCEAN at 6001 N Ocean Dr #7...
4,PAST SALE,2024-10-18,Condo/Co-op,3111 N Ocean Dr #1107,Hollywood,FL,33019.0,925000.0,3.0,2.0,Hollywood Towers,1650.0,NaN,1967.0,NaN,561.0,1214.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3111-N-Oce...,Beaches MLS,F10455532,N,Y,26.030688,-80.115514,https://www.redfin.com/FL/Hollywood/3111-N-Oce...,Lilli Schipper,LoKation,None,None,Rachael Osburn,Sunshine Realty,None,None,5,blue,Hollywood Towers at 3111 N Ocean Dr #1107 in H...
5,PAST SALE,2024-10-16,Condo/Co-op,3100 NE 48th St Ph 13,Fort Lauderdale,FL,33308.0,700000.0,2.0,2.0,Pilot House,1544.0,NaN,1977.0,NaN,453.0,1138.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10413857,N,Y,26.188219,-80.103988,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Timothy Ryan,Galt Ocean Realty Inc,None,None,Timothy Ryan,Galt Ocean Realty Inc,None,None,6,blue,Pilot House at 3100 NE 48th St Ph 13 in Fort L...
6,PAST SALE,2024-10-18,Condo/Co-op,4111 S Ocean Dr #705,Hollywood,FL,33019.0,635000.0,1.0,1.0,4111 SOUTH OCEAN DRIVE CO,701.0,NaN,2017.0,NaN,906.0,955.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/4111-S-Oce...,MARMLS,A11573649,N,Y,25.986806,-80.118650,https://www.redfin.com/FL/Hollywood/4111-S-Oce...,Jennifer Leong,One Sotheby's Int'l Realty,Mareste Contreras,One Sotheby's Int'l Realty,Mareste Contreras,One Sotheby's Int'l Realty,None,None,7,blue,4111 SOUTH OCEAN DRIVE CO at 4111 S Ocean Dr #...
7,PAST SALE,2024-10-16,Condo/Co-op,4111 S Ocean Dr #1412,Hollywood,FL,33019.0,619000.0,1.0,1.0,4111 SOUTH OCEAN DRIVE CO,869.0,NaN,2017.0,NaN,712.0,1191.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/4111-S-Oce...,MARMLS,A11530514,N,Y,25.986806,-80.1186

In [54]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: SABBIA BEACH CONDO closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $575,000M to $1,720,000M
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Broward condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Fort Lauderdale, Pompano Beach, Hollywood, Weston

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 111 condo sales totaling $33,541,399 million from Oct. 13th to Oct. 20th. The previous week, brokers closed 70 condo sales totaling $20,600,000.

Last week’s units sold for an average of $302,175, higher than the $294,649 average price from the previous week. The average price per square foot rose to $276

In [61]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Pompano-Beach/730-N-Ocean-Blvd-33062/unit-505/home/114019647


In [62]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Fort-Lauderdale/100-E-Las-Olas-Blvd-33301/unit-1701/home/160370571


In [63]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Fort-Lauderdale/3025-Harbor-Dr-33316/unit-17/home/41799320


In [64]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Weston/16100-Emerald-Estates-Dr-33331/unit-483/home/42063445


In [65]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Pompano-Beach/730-N-Ocean-Blvd-33062/unit-505/home/114019647


In [66]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Weston/16100-Emerald-Estates-Dr-33331/unit-483/home/42063445


In [67]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-18,Condo/Co-op,730 N Ocean Blvd #505,Pompano Beach,FL,33062.0,1720000.0,3.0,3.0,SABBIA BEACH CONDO,1830.0,NaN,2019.0,NaN,940.0,2997.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/730-N-...,MARMLS,A11619734,N,Y,26.243932,-80.087321,https://www.redfin.com/FL/Pompano-Beach/730-N-...,Daniel Mahaney,RealLook.com,None,None,None,None,None,None,1,orange,SABBIA BEACH CONDO at 730 N Ocean Blvd #505 in...
1,PAST SALE,2024-10-15,Condo/Co-op,100 E Las Olas Blvd #1701,Fort Lauderdale,FL,33301.0,1625000.0,3.0,2.5,100 E Las Olas,2395.0,NaN,2020.0,NaN,678.0,2792.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Beaches MLS,F10455449,N,Y,26.118835,-80.141476,https://www.redfin.com/FL/Fort-Lauderdale/100-...,Peter Barkin,"Compass Florida, LLC",Johnny Ramirez,"Compass Florida, LLC",Adriana Resendez,Coldwell Banker Realty,None,None,2,blue,100 E Las Olas at 100 E Las Olas Blvd #1701 in...
2,PAST SALE,2024-10-18,Condo/Co-op,3400 Galt Ocean Dr Unit 501S,Fort Lauderdale,FL,33308.0,1212500.0,2.0,2.5,SOUTHPOINT CONDO,1891.0,NaN,1974.0,NaN,641.0,1468.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3400...,MARMLS,A11657484,N,Y,26.169887,-80.098341,https://www.redfin.com/FL/Fort-Lauderdale/3400...,Sara Arroyo,"Compass Florida, LLC",Megan Probst,"Compass Florida, LLC",None,None,None,None,3,blue,SOUTHPOINT CONDO at 3400 Galt Ocean Dr Unit 50...
3,PAST SALE,2024-10-17,Condo/Co-op,6001 N Ocean Dr #702,Hollywood,FL,33019.0,940000.0,2.0,2.5,RENAISSANCE ON THE OCEAN,2025.0,NaN,1999.0,NaN,464.0,2316.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/6001-N-Oce...,MARMLS,A11584064,N,Y,26.051895,-80.112743,https://www.redfin.com/FL/Hollywood/6001-N-Oce...,Nina Spinner-Sands,The Keyes Company,Rita Morse,The Keyes Company,Florencia Baques Landry,"BB Realty Group, LLC.",None,None,4,blue,RENAISSANCE ON THE OCEAN at 6001 N Ocean Dr #7...
4,PAST SALE,2024-10-18,Condo/Co-op,3111 N Ocean Dr #1107,Hollywood,FL,33019.0,925000.0,3.0,2.0,Hollywood Towers,1650.0,NaN,1967.0,NaN,561.0,1214.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3111-N-Oce...,Beaches MLS,F10455532,N,Y,26.030688,-80.115514,https://www.redfin.com/FL/Hollywood/3111-N-Oce...,Lilli Schipper,LoKation,None,None,Rachael Osburn,Sunshine Realty,None,None,5,blue,Hollywood Towers at 3111 N Ocean Dr #1107 in H...
5,PAST SALE,2024-10-16,Condo/Co-op,3100 NE 48th St Ph 13,Fort Lauderdale,FL,33308.0,700000.0,2.0,2.0,Pilot House,1544.0,NaN,1977.0,NaN,453.0,1138.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Beaches MLS,F10413857,N,Y,26.188219,-80.103988,https://www.redfin.com/FL/Fort-Lauderdale/3100...,Timothy Ryan,Galt Ocean Realty Inc,None,None,Timothy Ryan,Galt Ocean Realty Inc,None,None,6,blue,Pilot House at 3100 NE 48th St Ph 13 in Fort L...
6,PAST SALE,2024-10-18,Condo/Co-op,4111 S Ocean Dr #705,Hollywood,FL,33019.0,635000.0,1.0,1.0,4111 SOUTH OCEAN DRIVE CO,701.0,NaN,2017.0,NaN,906.0,955.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/4111-S-Oce...,MARMLS,A11573649,N,Y,25.986806,-80.118650,https://www.redfin.com/FL/Hollywood/4111-S-Oce...,Jennifer Leong,One Sotheby's Int'l Realty,Mareste Contreras,One Sotheby's Int'l Realty,Mareste Contreras,One Sotheby's Int'l Realty,None,None,7,blue,4111 SOUTH OCEAN DRIVE CO at 4111 S Ocean Dr #...
7,PAST SALE,2024-10-16,Condo/Co-op,4111 S Ocean Dr #1412,Hollywood,FL,33019.0,619000.0,1.0,1.0,4111 SOUTH OCEAN DRIVE CO,869.0,NaN,2017.0,NaN,712.0,1191.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/4111-S-Oce...,MARMLS,A11530514,N,Y,25.986806,-80.1186

## Time on Market Calculator

In [69]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 8, 9) ## List (Earlier) date
date2 = datetime(2024, 10, 15) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

67


## Clean CSV for Datawrapper Chart

In [70]:
chart_df = df_top_ten

In [71]:
chart_df = chart_df.fillna(" ")

In [72]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [73]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [74]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [75]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [76]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [77]:
csv_date_string = today.strftime("%m_%d_%Y")

In [78]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")